In [2]:
!pip install datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00


In [3]:
import torch
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer
from datasets import load_dataset
import evaluate

In [5]:
# Load Model & Tokenizer
model_name = "deepset/roberta-base-squad2"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [6]:
# Load SQuAD v2 Validation Set
dataset = load_dataset("squad_v2", split="validation")

# QA Pipeline for Prediction
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Load Evaluation Metrics
squad_metric = evaluate.load("squad_v2")



README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Device set to use cuda:0


In [7]:
# Prepare Predictions
predictions = []
references = []

for example in dataset.select(range(10000)):  # Limit to 100 samples for quick testing
    context = example["context"]
    question = example["question"]
    ground_truths = example["answers"]["text"]

    # Handle empty ground truth cases
    if len(ground_truths) == 0:
        ground_truths = [""]

    # Get Model Prediction
    pred = qa_pipeline({"context": context, "question": question})
    pred_text = pred["answer"]

    # Append to Evaluation Lists
    predictions.append({
        "id": example["id"],
        "prediction_text": pred_text,
        "no_answer_probability": 0.0  # 
    })


    references.append({
        "id": example["id"],
        "answers": {
            "text": ground_truths,
            "answer_start": [0] * len(ground_truths)  # 
        }
    })

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [8]:
# Debug: Print one sample
print("Sample Prediction:", predictions[902])
print("Sample Reference:", references[902])

Sample Prediction: {'id': '570611c475f01819005e793c', 'prediction_text': 'Port of Los Angeles', 'no_answer_probability': 0.0}
Sample Reference: {'id': '570611c475f01819005e793c', 'answers': {'text': ['Port of Los Angeles', 'the Port of Los Angeles', 'Port of Los Angeles'], 'answer_start': [0, 0, 0]}}


In [9]:
# Compute Exact Match (EM) & F1 Score
results = squad_metric.compute(predictions=predictions, references=references)

In [10]:

# Print Results
print(f"Exact Match (EM): {results['exact']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")

Exact Match (EM): 42.03
F1 Score: 45.69
